# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [6]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [9]:
ed = Website("https://edwarddonner.com")
ed.links
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [10]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [11]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [12]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [13]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [14]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [15]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/openai/gpt-oss-120b',
 '/openai/gpt-oss-20b',
 '/Qwen/Qwen-Image',
 '/zai-org/GLM-4.5V',
 '/rednote-hilab/dots.ocr',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen-Image',
 '/spaces/amd/gpt-oss-120b-chatbot',
 '/spaces/black-forest-labs/FLUX.1-Krea-dev',
 '/spaces/ZhengGeng/OnePoseviaGen',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/jxm/gpt-oss20b-samples',
 '/datasets/nvidia/Llama-Nemotron-VLM-Dataset-v1',
 '/datasets/allenai/WildChat-4.8M',
 '/datasets/HuggingFaceH4/Multilingual-Thinking',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffuser

In [16]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'community - Discord',
   'url': 'https://huggingface.co/join/discord'},
  {'type': 'product page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [17]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [18]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Community page', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
openai/gpt-oss-120b
Updated
about 10 hours ago
•
490k
•
3.28k
openai/gpt-oss-20b
Updated


In [27]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'company homepage', 'url': 'https://huggingface.co/'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nopenai/gpt-oss-120b\nUpdated\nabout 10 hours ago\n•\n490k\n•\n3.28k\nopenai/gpt-oss-20b\nUpdated\nabout 10 hours ago\n•\n2.37M\n•\n2.86k\nQwen/Qwen-Image\nUpdated\n7 days ago\n•\n69.7k\n•\n1.54k\nzai-org/GLM-4.5V\nUpdated\n1 day ago\n•\n622\n•\n362\nrednote-hilab/dots.ocr\nUpdated\n1 day ago\n•\n17.9k\n•\n656\nBrowse 1M+ models\nSpaces\nRunning\n11.8k\n11.8k\nDeepSite v2\n🐳\nGenerate any a

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'brand / about page', 'url': 'https://huggingface.co/brand'}, {'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'Community / Forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'Status', 'url': 'https://status.huggingface.co/'}, {'type': 'Product: Endpoints', 'url': 'https://endpoints.huggingface.co'}, {'type': 'Blog', 'url': 'https://huggingface.co/blog'}]}


# Hugging Face — The AI community building the future

The platform where the machine learning community collaborates on models, datasets, and applications.

---

## About Hugging Face

- The collaboration platform for the machine learning community.
- The Hub is a central place to share, explore, discover, and experiment with open-source ML.
- Hugging Face empowers the next generation of ML engineers, scientists, and end users to learn, collaborate, and share their work to build an open and ethical AI future together.

 Tagline: The AI community building the future.

---

## What you can do on Hugging Face

- Move faster with the HF Open Source stack.
- Explore all modalities: text, image, video, audio, or even 3D.
- Build your portfolio: share your work with the world and build an ML profile.
- Host and collaborate on unlimited public models, datasets, and applications.
- Sign up to join a thriving community and start contributing.

---

## Core offerings

- Models: Browse 1M+ models
- Datasets: Browse 250k+ datasets
- Spaces: Applications and demos you can run or build
- Community, Docs, Enterprise, Pricing
- Sign Up to get started

Accelerate your ML with paid compute and enterprise solutions:
- Compute: Deploy on optimized Inference Endpoints or upgrade your Spaces to GPU in a few clicks. Starting at $0.60/hour for GPU.
- Enterprise: Team access with enterprise-grade security, access controls, and dedicated support. Starting at $20/user/month.
- Features: Single Sign-On (SSO), regional support, priority support, audit logs, resource groups, and private datasets viewer.

More than 50,000 organizations rely on Hugging Face, including major industry and research players.

---

## Open Source foundations powering the platform

Hugging Face is built on a robust open-source stack that drives both research and production:

- Transformers — State-of-the-art AI models for PyTorch
- Diffusers — Diffusion models for PyTorch
- Safetensors — Safe, efficient storage/distribution of weights
- Hub Python Library — Python client to interact with the Hugging Face Hub
- Tokenizers — Fast tokenizers for research and production
- TRL — Training transformers with reinforcement learning
- Transformers.js — Run transformers directly in the browser
- smolagents — Lightweight agents for Python
- PEFT — Parameter-efficient fine-tuning
- Datasets — Access and share datasets for ML tasks
- Text Generation Inference — Serve language models with TGI toolkit
- Accelerate — Train PyTorch models with multi-GPU/TPU/mixed precision

---

## Community & culture

- Open source, collaborative, and community-driven by design.
- Encourages learning, sharing, and ethical AI development.
- A space where researchers, engineers, and end users alike contribute to and benefit from cutting-edge ML tooling.
- Brand ethos emphasizes building an open and ethical AI future together.

---

## Our customers & partners

Hugging Face serves a broad set of organizations across research and industry. Examples include:

- AI2
- AI at Meta
- Amazon
- Google
- Intel
- Microsoft
- Grammarly
- Writer

These adopters illustrate the platform’s versatility—from research-centric models to enterprise-scale deployments.

---

## Careers at Hugging Face

Hugging Face maintains a dedicated Careers page with current openings. If you’re passionate about open science, collaboration, and building tools that empower the ML community, check the Careers page for opportunities to join the team.

Note: Specific job listings aren’t listed here, but the brand emphasizes opportunities to contribute to the foundation of ML tooling with the community.

---

## Brand essence

- Hugging Face positions itself as the collaboration platform for the machine learning community.
- The mission is to empower ML engineers, scientists, and end users to learn, collaborate, and share their work.
- The goal is to build an open and ethical AI future together.

---

## Quick-start suggestions

- Sign up and start exploring 1M+ models and 250k+ datasets.
- Browse or run Spaces to see real applications in action.
- Explore open-source tools to power your own ML projects (Transformers, Diffusers, Datasets, etc.).
- If you’re part of an organization seeking scalable AI capabilities, review Compute and Enterprise options.

---

### Want to learn more?
Visit Hugging Face to explore models, datasets, Spaces, and enterprise options, or head to the brand/assets and careers pages to understand culture, opportunities, and the broader mission.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'brand/about page', 'url': 'https://huggingface.co/brand'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'API endpoints', 'url': 'https://endpoints.huggingface.co'}]}


# Hugging Face: The AI community building the future

The party where models, datasets, and apps mingle — and you’re invited.

---

## Why Hugging Face?

- The home for collaboration in ML: host and work with unlimited public models, datasets, and applications.
- One platform for all modalities: text, image, video, audio, and even 3D.
- An open-source mindset that moves you faster with an open stack you can actually trust.

---

## What we offer

- The Home of Machine Learning
  - Create, discover, and collaborate on ML projects.
  - Explore 1M+ models and 250k+ datasets.
  - Build and share your work to grow your ML portfolio.

- Core platforms
  - Models: A vast model zoo with examples like openai/gpt-oss-120b, openai/gpt-oss-20b, Qwen/Qwen-Image, and more.
  - Datasets: A growing catalog including fka/awesome-chatgpt-prompts, jxm/gpt-oss20b-samples, and others.
  - Spaces: Run demos and deploy apps — e.g., DeepSite v2, Qwen Image, GPT-OSS-120B on AMD MI300X, and more.
  - Community & Docs: Learn, discuss, and contribute with a thriving ecosystem.

- Open Source Stack (the heart of our toolkit)
  - Transformers, Diffusers, Safetensors, Tokenizers, TRL, PEFT, and more.
  - Hub Python Library to interact with the Hugging Face Hub.
  - Transformers.js to run models in your browser.
  - Smolagents for building smart agents in Python.
  - A robust ecosystem designed for research and production.

- Enterprise & Compute
  - Compute: Deploy on optimized Inference Endpoints or upgrade Spaces apps to GPUs with ease.
  - Pricing starting at $0.60/hour for GPU.
  - Team & Enterprise: enterprise-grade security, access controls, and dedicated support.
  - Getting started: starting at $20 per user per month, with SSO, regions, audit logs, resource groups, and private datasets viewer.

---

## Who’s using Hugging Face?

- 50,000+ organizations rely on Hugging Face.
- Notable names include AI2, Meta, Amazon, Google, Intel, Microsoft, Grammarly, Writer, and more.
- A thriving ecosystem of models, datasets, and apps that keeps growing every day.

---

## The culture (in real talk)

- Open, collaborative, and community-driven: we’re built by and for the community — researchers, developers, and enthusiasts who love sharing what they build.
- Multimodal by design: text, image, video, audio, and 3D — because real-world problems don’t come in neat little boxes.
- Transparent and fast-moving: a hub for ideas, contributions, and rapid iteration.
- Supportive growth: tools, docs, and forums to help you contribute, learn, and shine.

If you want to contribute, ship features, or just nerd out about AI with friendly folks, you’ll fit right in.

---

## Careers & getting involved

- Careers: We have a dedicated Jobs page and a culture of contribution and learning. Whether you’re a researcher, engineer, or product-minded builder, there’s a place for you.
- Where to look: Explore opportunities on our Jobs page and get involved in our community initiatives, documentation, and open-source projects.
- Why join: work on the cutting edge of ML tooling with a global community, flexible collaboration tools, and enterprise-grade security when you need it.

---

## Get started

- Sign up or log in to join the AI community building the future.
- Browse 1M+ models, 250k+ datasets, and a growing set of Spaces to spark your next project.
- Sign up for enterprise options or start with free compute to test ideas.

Explore. Collaborate. Create. Leave your mark on the future of AI with Hugging Face.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>